In [ ]:
import time

def generate_instances(filename, cnt, N, p, q, T):
    if cnt > 10**7:
        cnt = 10**7

    try:
        with open(filename, 'a') as file:
            print(f"Successfully opened file: {filename}")
            
            start_time = time.time()
            # will output a message every time 10% get generated
            block = 8500
            
            R = Zmod(N);
            
            for i in range(cnt):
                #Generate random x and compute the corresponding y
                x = R.random_element()
                y = power_mod(x, (2^T) % phi, N)
                
                file.write(str(x) + '\n' + str(y) + '\n')
                
                if i % block == 0:
                    current_time = time.time()
                    elapsed_time = current_time - start_time
                    print(f"Generated {i} instances in {elapsed_time} seconds")
                
                i += 1

    except IOError:
        print(f"Failed to open file: {filename}")
        return [], []
    
    return True

In [ ]:
def get_instances(file_name, cnt):
    if cnt > 10**7:
        cnt = 10**7

    cur_x = []
    cur_y = []

    try:
        with open(file_name, 'r') as file:
            print(f"Successfully opened file: {file_name}")

            i = 0
            for line in file:
                if i >= cnt * 2:
                    break

                # Parse the line to a SageMath integer
                cur_num = Integer(line.strip())

                if i % 2 == 0:
                    if cur_num == 0:
                        print("Current x is 0, skipping it!")
                        next(file)  # Skip the next line
                        continue
                    cur_x.append(cur_num)
                else:
                    cur_y.append(cur_num)

                i += 1

    except IOError:
        print(f"Failed to open file: {file_name}")
        return [], []
    
    if len(cur_x) < cnt:
        while len(cur_x) < cnt:
            cur_x.extend(cur_x)
            cur_y.extend(cur_y)
        cur_x = cur_x[:cnt]
        cur_y = cur_y[:cnt]
    
    return cur_x, cur_y

In [ ]:
# Generate a random positive integer with l bits
def get_int(l):
    if l <= 0:
        raise ValueError("The bit length must be a positive integer")
    
    random_num = randint(0, 2**l - 1)

    return random_num

def get_alphas(cnt, l):
    if cnt <= 0:
        raise ValueError("The count must be a positive integer")
    return [get_int(l) for _ in range(cnt)]

def get_buckets(cnt, k):
    if cnt <= 0:
        raise ValueError("The count must be a positive integer")
    return [get_int(k) for _ in range(cnt)]

In [ ]:
def combine_Exponents(x, y, N, l):
    if not (len(x) == len(y)):
        raise ValueError("Lists x, y, and alpha must be of the same length")

    # Ensure N is a positive integer
    if N <= 0:
        raise ValueError("N must be a positive integer")
        
    alpha = get_alphas(len(x), l)

    # Compute cx as the product of all x[i] raised to alpha[i], modulo N
    # Compute cy in a similar manner
    cx = 1
    cy = 1
    for i in range(len(x)):
        cx = (cx * power_mod(x[i], alpha[i], N)) % N
        cy = (cy * power_mod(y[i], alpha[i], N)) % N
        
    return cx, cy

In [ ]:
def combine_Subsets(x, y, N, l):
    if not (len(x) == len(y)):
        raise ValueError("Lists x, y, and alpha must be of the same length")

    # Ensure N is a positive integer
    if N <= 0:
        raise ValueError("N must be a positive integer")

    cx = []
    cy = []    
    for _ in range(l):
        alpha = get_alphas(len(x), 1)
        # Compute cx as the product of all x[i] selected by alpha[i], modulo N
        # Compute cy in a similar manner
        curr_x = 1
        curr_y = 1
        for i in range(len(x)):
            if alpha[i] == 1:
                curr_x = (curr_x * x[i]) % N
                curr_y = (curr_y * y[i]) % N

        cx.append(curr_x)
        cy.append(curr_y)
    
    return cx, cy

In [ ]:
def combine_Hybrid(x, y, N, l, k):
    if not (len(x) == len(y)):
        raise ValueError("Lists x, y, and alpha must be of the same length")

    # Ensure N is a positive integer
    if N <= 0:
        raise ValueError("N must be a positive integer")

    cx = []
    cy = []    
    for _ in range(l):
        alpha = get_alphas(len(x), 1)
        # Compute cx as the product of all x[i] selected by alpha[i], modulo N
        # Compute cy in a similar manner
        curr_x = 1
        curr_y = 1
        for i in range(len(x)):
            if alpha[i] == 1:
                curr_x = (curr_x * x[i]) % N
                curr_y = (curr_y * y[i]) % N

        cx.append(curr_x)
        cy.append(curr_y)
        
    final_x, final_y = combine_Exponents(cx, cy, N, k)
    
    return final_x, final_y

In [ ]:
def optimalK(cnt, l):
    # Initialize variables to store the minimum value and corresponding k
    min_value = infinity
    optimal_k = 3

    # Iterate over possible values of k
    for k in range(3, l + 1):
        current_value = ceil(QQ(l) / (k - 2)) * (2 * cnt + (3 * k + 2) * 2^k + (3 * l + 2))
        if current_value < min_value:
            min_value = current_value
            optimal_k = k

    return optimal_k    

def combine_Bucket(x, y, N, l):
    if not (len(x) == len(y)):
        raise ValueError("Lists x, y, and alpha must be of the same length")

    # Ensure N is a positive integer
    if N <= 0:
        raise ValueError("N must be a positive integer")

    cnt = len(x)
    k = optimalK(cnt, l)
    rho = ceil(QQ(l) / (k - 2))
    
    
    rho_x = []
    rho_y = []
    
    for _ in range(rho):
        buckets = get_buckets(cnt,k)
        cx = [1 for _ in range(2**k)]
        cy = [1 for _ in range(2**k)]
        for i in range(cnt):
            cx[buckets[i]] = (cx[buckets[i]] * x[i]) % N
            cy[buckets[i]] = (cy[buckets[i]] * y[i]) % N

        curr_x, curr_y = combine_Exponents(cx,cy, N, k)

        rho_x.append(curr_x)
        rho_y.append(curr_y)
        
    final_x, final_y = combine_Exponents(rho_x, rho_y, N, l)
    
    return final_x, final_y

In [ ]:
N = 22271705427910772334877681575019141239360195315023327731872705030553377327979866044243199497323528404368833049911941802832106863948559802495321282883008184358992335728090241285890650336285197555442679460021182578675760887828777456097537051796685338914270253532506593869283213790271854177668295279903868385435684092618306195602938458695163944131069634975226554436302840338113082478664088927606478243501161955441052266600616420466099382013651807480774112937376017734586371438278013241438847012572310186295364466203780870693111359678214315882481986344724840584694166335743896271129684073568128960223634580483449510308341;
p = 142835209786489997170873255507946573556448214084628524605421188224099687322984673543614176852071053547415698252622573845683075452982474922517841367278609089136988583447756479483918441795733215771335093892746851604252279730368411597397577626119447392355080344631875081748708394736819710836176156337925349997841;
q = 155925877528394482614610625993674588019100771066359218078554587162570881239567576804461126115887903799308419845098579180815670096035521874870908961799638269196068560103752308669818128860677719460381304397587862593607968358286567068579479763671817955144283945749615768573725580291910494735428411976050787790501;
l = 128;
phi = (p-1)*(q-1);
T = 2**25;
filename = 'instances-new.txt'

In [ ]:
cnt = 10000000;

In [ ]:
generate_instances(filename, cnt, N, p, q, T)

In [ ]:
for cnt in [10**t for t in range(2,8)]:
    x,y = get_instances(filename, cnt);

    # Runs the code 10 times and gives the best time
    time = timeit('cx, cy = combine_Exponents(x,y,N,l)', number=10)
    print('EXPONENTS on ' + str(cnt) + ' instances: ' + str(time))
    
    # Runs the code 10 times and gives the best time
    time = timeit('cx, cy = combine_Subsets(x,y,N,l)', number=10)
    print('SUBSETS on ' + str(cnt) + ' instances: ' + str(time))
    
    # Runs the code 10 times and gives the best time
    time = timeit('cx, cy = combine_Hybrid(x, y, N, l, l)', number=10)
    print('HYBRID on ' + str(cnt) + ' instances: ' + str(time))
    
    # Runs the code 10 times and gives the best time
    time = timeit('cx, cy = combine_Bucket(x, y, N, l)', number=10)
    print('BUCKET on ' + str(cnt) + ' instances: ' + str(time))